DEPUIS MICE

In [30]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import os


In [31]:
# 1. Charger un dataset déjà imputé (ex. par KNN ou MICE)
df = pd.read_csv("C:/Users/bamoi/OneDrive - Groupe ESAIP/$ PSIR/experimentation/repo/psir-ml-imputation/src/imputation/hta_impute_MICE.csv")  # tu peux changer par KNN aussi



In [32]:
# 2. Séparer les features et la cible
X = df.drop(columns=["HTA"])
y = df["HTA"]



In [33]:
# 3. Séparer en train/test avant SMOTE (important !)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)



One-Hot Encoding sur les colonnes non numériques

In [34]:
cat_cols = X_train.select_dtypes(include="object").columns.tolist()
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
encoder.fit(X_train[cat_cols])

print("🧠 One-hot encoding des colonnes catégorielles :", cat_cols)
X_train_encoded = encoder.fit_transform(X_train[cat_cols])
X_test_encoded = encoder.transform(X_test[cat_cols])

encoded_col_names = encoder.get_feature_names_out(cat_cols)

🧠 One-hot encoding des colonnes catégorielles : ['groupe_age']


Numériques bruts

In [35]:
X_train_num = X_train.drop(columns=cat_cols).reset_index(drop=True)
X_test_num = X_test.drop(columns=cat_cols).reset_index(drop=True)

Recomposer les matrices complètes encodées

In [36]:
X_train_full = pd.concat([
        X_train_num,
        pd.DataFrame(X_train_encoded, columns=encoded_col_names)
    ], axis=1)

X_test_full = pd.concat([
        X_test_num,
        pd.DataFrame(X_test_encoded, columns=encoded_col_names)
    ], axis=1)

In [37]:
# 4. Appliquer SMOTE uniquement sur le training set
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_smote, y_train_smote = smote.fit_resample(X_train_full, y_train)

In [38]:
# 5. Vérifier la nouvelle distribution des classes
print("📊 Distribution des classes après SMOTE :")
print(y_train_smote.value_counts())



📊 Distribution des classes après SMOTE :
HTA
False    9070
True     9070
Name: count, dtype: int64


In [ ]:
# 6. Sauvegarder les datasets équilibrés (facultatif mais conseillé)
pd.DataFrame(X_train_smote, columns=X_train_full.columns).assign(HTA=y_train_smote).to_csv("train_SMOTE_MICE.csv", index=False)
pd.DataFrame(X_test_full, columns=X_test_full.columns).assign(HTA=y_test.values).to_csv("test_original_MICE.csv", index=False)
print("✅ Datasets équilibrés sauvegardés : train_SMOTE.csv et test_original.csv")

✅ Datasets équilibrés sauvegardés : train_SMOTE.csv et test_original.csv


# DEPUIS KNN

In [40]:
# 1. Charger un dataset déjà imputé (ex. par KNN ou MICE)
df = pd.read_csv("C:/Users/bamoi/OneDrive - Groupe ESAIP/$ PSIR/experimentation/repo/psir-ml-imputation/src/imputation/hta_impute_KNN.csv")  # tu peux changer par KNN aussi

In [41]:
# 2. Séparer les features et la cible
X = df.drop(columns=["HTA"])
y = df["HTA"]

In [42]:
# 3. Séparer en train/test avant SMOTE (important !)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

One-Hot Encoding sur les colonnes non numériques

In [43]:
cat_cols = X_train.select_dtypes(include="object").columns.tolist()
encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")

print("🧠 One-hot encoding des colonnes catégorielles :", cat_cols)
X_train_encoded = encoder.fit_transform(X_train[cat_cols])
X_test_encoded = encoder.transform(X_test[cat_cols])

encoded_col_names = encoder.get_feature_names_out(cat_cols)

🧠 One-hot encoding des colonnes catégorielles : ['groupe_age']


Numériques bruts

In [44]:
X_train_num = X_train.drop(columns=cat_cols).reset_index(drop=True)
X_test_num = X_test.drop(columns=cat_cols).reset_index(drop=True)

Recomposer les matrices complètes encodées

In [45]:
X_train_full = pd.concat([
        X_train_num,
        pd.DataFrame(X_train_encoded, columns=encoded_col_names)
    ], axis=1)

X_test_full = pd.concat([
        X_test_num,
        pd.DataFrame(X_test_encoded, columns=encoded_col_names)
    ], axis=1)

In [46]:

# 4. Appliquer SMOTE uniquement sur le training set
smote = SMOTE(random_state=42, k_neighbors=5)
X_train_smote, y_train_smote = smote.fit_resample(X_train_full, y_train)



In [47]:
# 5. Vérifier la nouvelle distribution des classes
print("📊 Distribution des classes après SMOTE :")
print(y_train_smote.value_counts())



📊 Distribution des classes après SMOTE :
HTA
False    9070
True     9070
Name: count, dtype: int64


In [48]:
# 6. Sauvegarder les datasets équilibrés (facultatif mais conseillé)
pd.DataFrame(X_train_smote, columns=X.columns).assign(HTA=y_train_smote).to_csv("train_SMOTE_KNN.csv", index=False)
pd.DataFrame(X_test, columns=X.columns).assign(HTA=y_test).to_csv("test_original_KNN.csv", index=False)
print("✅ Datasets équilibrés sauvegardés : train_SMOTE.csv et test_original.csv")

✅ Datasets équilibrés sauvegardés : train_SMOTE.csv et test_original.csv
